In [1]:
pip install torchcp

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.1 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import argparse
import os

import torch
import torchvision
import torchvision.datasets as dset
from torchvision import transforms
from tqdm import tqdm

from torchcp.classification.predictors import ClusterPredictor, ClassWisePredictor, SplitPredictor
from torchcp.classification.scores import THR, APS, SAPS, RAPS
from torchcp.classification import Metrics
from torchcp.utils import fix_randomness
from examples.common.dataset import build_dataset

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


# 参数设置

In [3]:
alpha = 0.1

# 加载设置模型

In [4]:
model_name = 'resnet50'
model = torchvision.models.resnet50(weights="IMAGENET1K_V1", progress=True)
model_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(model_device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# 数据集选择

## MNIST

In [51]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x:x.repeat(3,1,1))
])
dataset = torchvision.datasets.MNIST(root='/shareddata', download=True,train=True,transform = transform)
cal_dataset, test_dataset = torch.utils.data.random_split(dataset, [30000, 30000])#两个数字之和要等于数据集总长度
cal_data_loader = torch.utils.data.DataLoader(cal_dataset, batch_size=1024, shuffle=False, pin_memory=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False, pin_memory=True)

## Flowers102

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小
    transforms.ToTensor(),           # 将图像转换为张量
])

dataset = torchvision.datasets.Flowers102(root='/home/', download=True,split='train',transform = transform)

train_ratio = 0.5  # 训练集占50%
test_ratio = 1 - train_ratio  # 测试集占50%

# 计算划分的样本数量
num_train = int(len(dataset) * train_ratio)
num_test = len(dataset) - num_train

# 划分数据集
cal_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_test])

cal_data_loader = torch.utils.data.DataLoader(cal_dataset, batch_size=1024, shuffle=False, pin_memory=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False, pin_memory=True)

100%|██████████| 344862509/344862509 [01:46<00:00, 3248015.67it/s]


Extracting /home/flowers-102/102flowers.tgz to /home/flowers-102


100%|██████████| 502/502 [00:00<00:00, 365164.86it/s]


100%|██████████| 14989/14989 [00:00<00:00, 8516448.48it/s]


# EuroSAT

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小
    transforms.ToTensor(),           # 将图像转换为张量
])


dataset = ImageFolder(root='/openbayes/input/input0', transform=transform)

train_ratio = 0.5  # 训练集占50%
test_ratio = 1 - train_ratio  # 测试集占50%

# 计算划分的样本数量
num_train = int(len(dataset) * train_ratio)
num_test = len(dataset) - num_train

# 划分数据集
cal_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_test])

cal_data_loader = torch.utils.data.DataLoader(cal_dataset, batch_size=1024, shuffle=False, pin_memory=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False, pin_memory=True)

# CP scores

## THR

In [6]:
score_function = THR()

In [14]:
score_function = APS()

# Predictors

## SplitPredictor

In [15]:
predictor = SplitPredictor(score_function, model)

In [17]:
predictor = ClusterPredictor(score_function, model)

In [7]:
predictor = ClassWisePredictor(score_function, model)

# Result

In [ ]:
predictor.calibrate(cal_data_loader, alpha)
predictor.evaluate(test_data_loader)

/usr/local/lib/python3.8/site-packages/torchcp/classification/predictors/split.py:61: UserWarning: The number of scores is 0, which is a invalid scores. To avoid program crash, the threshold is set as torch.inf.
  warnings.warn(
